# TP Sytèmes de recommandation

## 1 Présentation du modèle

### Question 1.1

*Que fait l'option `minidata` de la fonction `load_movielens` de `movielensutils.py` ?*


In [ ]:
    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

L'option `minidata` réduit les données de travail aux notes des 100 premiers utilisateurs pour les 200 premiers films.

### Question 1.2

*Combien y a-t-il d'utilisateurs, de films référencés dans la base de données ? Quel est le nombre total de notes ?*

Il y a :
* 943 utilisateurs
* 1682 films
* 100 000 notes

### Question 1.3

*La fonction objectif est-elle convexe ?*

Soit $f$ la fonction objectif.  


*Quel est son gradient ?*

$$
\nabla{f}(P,Q) =
\begin{bmatrix}
\nabla{f_Q} \\
\nabla{f_P}
\end{bmatrix}
= \begin{bmatrix}
\mathbb{1}_K \circ (-Q^T(R - QP) + \rho P) \\
\mathbb{1}_K \circ (R-QP)(-P^T) + \rho Q)
\end{bmatrix}
$$

*Est-il lipschitzien ? Donner la constance de Lipschitz le cas échéant.*

** TODO **

## 2 Trouver $P$ quand $Q_0$ est fixé

### Question 2.1

*La fonction objectif $g$ est-elle convexe ?*

$g$ étant un cas particulier de la fonction objectif $f$ qui n'est pas convexe, $g$ n'est pas convexe non plus. (??? showerthought says no)

*Quel est son gradient ?*

$$
TODO
$$

### Question 2.2

In [1]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = 

    return val, grad_P

SyntaxError: invalid syntax (<ipython-input-1-9a0071409f67>, line 20)